In [19]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import network

from block import GENESIS_BLOCK
from helper import calculate_new_bits
from network import (
    NetworkEnvelope,
    VersionMessage,
)

### Exercise 1

Determine what this network message is:

`f9beb4d976657261636b000000000000000000005df6e0e2`

In [45]:
# Exercise 1

message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'

# convert to binary
# see what the bytes 4 through 16 aregi

In [46]:
from network import NetworkEnvelope
from io import BytesIO

message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'
stream = BytesIO(bytes.fromhex(message_hex))
envelope = NetworkEnvelope.parse(stream)
print(envelope.command)

b'verack'


In [22]:
print(envelope.payload)

b''


### Exercise 2

Write the `parse` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_parse`

In [23]:
@classmethod
    def parse(cls, s, testnet=False):
        '''Takes a stream and creates a NetworkEnvelope'''
        # check the network magic
        magic = s.read(4)
        if magic == b'':
            raise RuntimeError('Connection reset!')
        if testnet:
            expected_magic = TESTNET_NETWORK_MAGIC
        else:
            expected_magic = NETWORK_MAGIC
        if magic != expected_magic:
            raise RuntimeError('magic is not right {} vs {}'.format(magic.hex(), expected_magic.hex()))
        
        # command 12 bytes
        command = s.read(12)
        # strip the trailing 0's
        command = command.strip(b'\x00')
        
        # payload length 4 bytes, little endian
        payload_length = little_endian_to_int(s.read(4))
        
        # checksum 4 bytes, first four of hash256 of payload
        checksum = s.read(4)
        
        # payload is of length payload_length
        payload = s.read(payload_length)
        
        # verify checksum
        calculated_checksum = hash256(payload)[:4]
        if calculated_checksum != checksum:
            raise IOError('checksum does not match')
        # return an instance of the class
        return cls(command, payload, testnet=testnet)

IndentationError: unexpected indent (<ipython-input-23-1ecbd183166b>, line 2)

In [47]:
# Exercise 2

reload(network)
run(network.NetworkEnvelopeTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 3

Write the `serialize` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_serialize`

In [48]:
# Exercise 3

reload(network)
run(network.NetworkEnvelopeTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


### Exercise 4

Write the `serialize` method for `VersionMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:VersionMessageTest:test_serialize`

In [44]:
# Exercise 4

reload(network)
run(network.VersionMessageTest("test_serialize"))

F
FAIL: test_serialize (network.VersionMessageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/kevin/gh/programmingbitcoin/code-ch10/network.py", line 215, in test_serialize
    self.assertEqual(v.serialize().hex(), '7f11010000000000000000000000000000000000000000000000000000000000000000000000ffff00000000208d000000000000000000000000000000000000ffff00000000208d0000000000000000182f70726f6772616d6d696e67626974636f696e3a302e312f0000000000')
AssertionError: '7f11[79 chars]000008d20000000000000000000000000000000000000f[87 chars]0000' != '7f11[79 chars]00000208d000000000000000000000000000000000000f[87 chars]0000'
Diff is 738 characters long. Set self.maxDiff to None to see it.

----------------------------------------------------------------------
Ran 1 test in 0.005s

FAILED (failures=1)


### Exercise 5

Write the `handshake` method for `SimpleNode`

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:SimpleNodeTest:test_handshake`

In [39]:
def handshake(self):
        '''Do a handshake with the other node.
        Handshake is sending a version message and getting a verack back.'''
        # create a version message
        version = VersionMessage()
        # send the command
        self.send(version)
        # wait for a verack message
        self.wait_for(VerAckMessage)

In [40]:
# Exercise 5

reload(network)
run(network.SimpleNodeTest("test_handshake"))

E
ERROR: test_handshake (network.SimpleNodeTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/kevin/gh/programmingbitcoin/code-ch10/network.py", line 391, in test_handshake
    node = SimpleNode('testnet.programmingbitcoin.com', testnet=True)
  File "/home/kevin/gh/programmingbitcoin/code-ch10/network.py", line 343, in __init__
    self.socket.connect((host, port))
ConnectionRefusedError: [Errno 111] Connection refused

----------------------------------------------------------------------
Ran 1 test in 0.112s

FAILED (errors=1)


### Exercise 6

Write the `serialize` method for `GetHeadersMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:GetHeadersMessageTest:test_serialize`

In [33]:
# Exercise 6

reload(network)
run(network.GetHeadersMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


In [34]:
from io import BytesIO
from network import SimpleNode, GetHeadersMessage, HeadersMessage
from block import Block, GENESIS_BLOCK, LOWEST_BITS
from helper import calculate_new_bits
previous = Block.parse(BytesIO(GENESIS_BLOCK))
first_epoch_timestamp = previous.timestamp
expected_bits = LOWEST_BITS
count = 1
node = SimpleNode('mainnet.programmingbitcoin.com', testnet=False)
node.handshake()
for _ in range(19):
    getheaders = GetHeadersMessage(start_block=previous.hash())
    node.send(getheaders)
    headers = node.wait_for(HeadersMessage)
    for header in headers.blocks:
        if not header.check_pow():
            raise RuntimeError('bad PoW at block {}'.format(count))
        if header.prev_block != previous.hash():
            raise RuntimeError('discontinuous block at {}'.format(count))
        if count % 2016 == 0:
            time_diff = previous.timestamp - first_epoch_timestamp
            expected_bits = calculate_new_bits(previous.bits, time_diff)
            print(expected_bits.hex())
            first_epoch_timestamp = header.timestamp
        if header.bits != expected_bits:
            raise RuntimeError('bad bits at block {}'.format(count))
        previous = header
        count += 1

NotImplementedError: 